
# Dynamic Factor Model (DFM) con VIX y MOVE (riesgo latente de mercado)

In [19]:
# 0) Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from statsmodels.tsa.statespace.dynamic_factor import DynamicFactor
from statsmodels.tsa.statespace.structural import UnobservedComponents

plt.rcParams["figure.figsize"] = (10, 4)



## 1) Datos

In [20]:
import pandas as pd
import yfinance as yf

def fetch_yahoo_index(ticker: str, start="1990-01-01", end=None) -> pd.Series:
    df = yf.download(ticker, start=start, end=end, progress=False, auto_adjust=False)
    if df.empty:
        raise ValueError(f"No data returned for {ticker}")

    # pick close column name that exists at top level
    if isinstance(df.columns, pd.MultiIndex):
        lvl0 = df.columns.get_level_values(0)
        price_col = "Adj Close" if "Adj Close" in lvl0 else "Close"
        px = df[price_col]  # usually a DataFrame with columns = tickers
        if isinstance(px, pd.DataFrame):
            px = px[ticker] if ticker in px.columns else px.iloc[:, 0]
    else:
        price_col = "Adj Close" if "Adj Close" in df.columns else "Close"
        px = df[price_col]  # Series

    px = px.dropna()
    px.name = ticker
    return px

move = fetch_yahoo_index("^MOVE", start="2000-01-01")
vix  = fetch_yahoo_index("^VIX",  start="1990-01-01")

iv = pd.concat([move, vix], axis=1).sort_index()
print(iv.tail())

                ^MOVE       ^VIX
Date                            
2026-02-13  70.099998  20.600000
2026-02-17  68.839996  20.290001
2026-02-18  64.190002  19.620001
2026-02-19  68.029999  20.230000
2026-02-20  64.269997  19.090000


In [21]:
df = iv.rename(columns={"^MOVE": "MOVE", "^VIX": "VIX"})

In [22]:
import plotly.graph_objects as go

fig = go.Figure()

for col in ["VIX", "MOVE"]:
    fig.add_trace(
        go.Scatter(
            x=df.index,
            y=df[col],
            mode="lines",
            name=col,
        )
    )

fig.update_layout(
    title="Series originales (niveles)",
    xaxis_title="Fecha",
    yaxis_title="Nivel",
    legend_title_text="Serie",
    template="plotly_white",
)

fig.show()


## 2) Transformaciones 

Para DFM con volatilidades implícitas, suele ser estable usar:
- $\log(\text{VIX})$, $\log(\text{MOVE})$
- y estandarizar (z-score) para que estén en escalas comparables.

Definimos:
$$
x_t = \begin{pmatrix} z(\log VIX_t) \\ z(\log MOVE_t) \end{pmatrix}
$$


In [23]:
# 2) Log + estandarización
X = np.log(df[["VIX", "MOVE"]]).copy()
X = (X - X.mean()) / X.std()  # z-scores

X.head()


,VIX,MOVE
Date,,
1990-01-02,-0.167750,NaN
1990-01-03,-0.010801,NaN
1990-01-04,0.150360,NaN
1990-01-05,0.282807,NaN
1990-01-08,0.304551,NaN


In [24]:
X = X.dropna()
X

,VIX,MOVE
Date,,
2002-11-12,1.509169,1.136079
2002-11-13,1.571648,1.202030
2002-11-14,1.320458,1.377817
2002-11-15,1.106680,1.424913
2002-11-18,1.215521,1.351202
...,...,...
2026-02-13,0.353247,-0.483019
2026-02-17,0.308880,-0.537471
2026-02-18,0.210630,-0.747432


In [34]:
X = X.copy().sort_index()
X.index = pd.to_datetime(X.index)
X = X[~X.index.duplicated(keep="first")]
X = X.asfreq("B")               # o la frecuencia correcta
# Decide UNA:
X = X.dropna()                # overlap estricto
# X = X.ffill()                 # panel completo imputado (ojo con sesgos)

In [35]:
Xz = (X - X.mean()) / X.std(ddof=0)
Xz = Xz.asfreq("B")


## 3) DFM con 1 factor (k=1) y dinámica AR(1) del factor

Modelo:

**Medición**
$$
x_t = \Lambda f_t + \varepsilon_t
$$

**Dinámica del factor**
$$
f_t = \phi f_{t-1} + u_t
$$

En `statsmodels`, esto se especifica como:
- `k_factors=1`
- `factor_order=1` (AR(1) para el factor)

El modelo estima:
- Cargas $\Lambda$ (sensibilidad de cada serie al factor).
- La trayectoria $f_t$ (filtrada y suavizada).


In [36]:
mod = DynamicFactor(Xz, k_factors=1, factor_order=2, error_order=1)
res = mod.fit(maxiter=2000, disp=False)

In [37]:
print(res.summary())

                                   Statespace Model Results                                  
Dep. Variable:                       ['VIX', 'MOVE']   No. Observations:                 6074
Model:             DynamicFactor(factors=1, order=2)   Log Likelihood                2969.996
                                      + AR(1) errors   AIC                          -5923.992
Date:                               Sat, 21 Feb 2026   BIC                          -5870.298
Time:                                       22:42:33   HQIC                         -5905.357
Sample:                                   11-12-2002                                         
                                        - 02-20-2026                                         
Covariance Type:                                 opg                                         
Ljung-Box (L1) (Q):          1023.12, 3.14   Jarque-Bera (JB):    12348.41, 7323.66
Prob(Q):                        0.00, 0.08   Prob(JB):                


## 4) Interpretación de $\Lambda$ (cargas)

En un DFM de 1 factor, $\Lambda$ es un vector con 2 entradas:
- $\lambda_{VIX}$: cuánto responde el VIX (transformado y estandarizado) al factor común.
- $\lambda_{MOVE}$: cuánto responde MOVE al factor común.

**Importante (identificación de signo):** el factor está definido hasta un signo. Si sale “invertido”, puedes multiplicar $f_t$ por $-1$ para que “riesgo alto” corresponda a VIX y MOVE altos.


In [39]:
# 4) Extraer cargas (Lambda) y factor (f_t)
# En statsmodels, los 'loadings' aparecen en params con prefijos.
params = res.params
params


loading.f1.VIX        2.762409e-01
loading.f1.MOVE       4.949466e-02
sigma2.VIX            4.078338e-11
sigma2.MOVE           1.477808e-02
L1.f1.f1              1.216821e-04
L2.f1.f1              9.998765e-01
L1.e(VIX).e(VIX)     -9.271580e-01
L1.e(MOVE).e(MOVE)    9.912314e-01
dtype: float64

In [ ]:


def factor_a_serie(factors_obj, which: str, index, k: int = 0) -> pd.Series:
    """
    Convierte res.factors.{filtered,smoothed} en pd.Series.
    Soporta outputs como DataFrame o ndarray con forma (T,k) o (k,T).
    """
    arr = getattr(factors_obj, which)

    # Caso 1: ya es DataFrame
    if isinstance(arr, pd.DataFrame):
        s = arr.iloc[:, k].copy()
        s.index = index
        s.name = f"factor_{k+1}_{which}"
        return s

    # Caso 2: ndarray
    arr = np.asarray(arr)
    T = len(index)

    if arr.ndim == 1:
        if arr.shape[0] != T:
            raise ValueError(f"Dimensión inesperada {arr.shape}; esperado largo T={T}.")
        return pd.Series(arr, index=index, name=f"factor_{k+1}_{which}")

    if arr.ndim != 2:
        raise ValueError(f"Dimensión inesperada {arr.ndim}; esperado 1 o 2.")

    # (T,k) vs (k,T)
    if arr.shape[0] == T:
        vec = arr[:, k]
    elif arr.shape[1] == T:
        vec = arr[k, :]
    else:
        raise ValueError(f"Forma inesperada {arr.shape}; no cuadra con T={T}.")

    return pd.Series(vec, index=index, name=f"factor_{k+1}_{which}")

# 4.1) Extraer series del factor: filtrado y suavizado
f_filt   = factor_a_serie(res.factors, "filtered", Xz.index, k=0)
f_smooth = factor_a_serie(res.factors, "smoothed", Xz.index, k=0)

In [54]:
s = f_smooth.rename("factor").to_frame().join(X[["VIX"]], how="inner").dropna()
rho = s["factor"].corr(s["VIX"])  # Pearson por defecto

if rho < 0:
    f_filt = -f_filt
    f_smooth = -f_smooth


In [56]:
df = (
    f_filt.rename("factor").to_frame()
    .join(X[["VIX", "MOVE"]], how="inner")
    .dropna()
)

corr_vix  = df["factor"].corr(df["VIX"])
corr_move = df["factor"].corr(df["MOVE"])

print(f"Corr(factor, VIX)  = {corr_vix:.6f}")
print(f"Corr(factor, MOVE) = {corr_move:.6f}")


Corr(factor, VIX)  = 1.000000
Corr(factor, MOVE) = 0.540775


In [61]:

fig = go.Figure(
    data=[
        go.Scatter(
            x=f_filt.index,
            y=f_filt.values,
            mode="lines",
            name="f_t",
        )
    ]
)

fig.update_layout(
    title="Factor latente: índice de riesgo común",
    xaxis_title="Fecha",
    yaxis_title="f_t (escala arbitraria)",
    template="plotly_white",
)

fig.show()


## 5) Tendencia del factor: $\tau_t$ y ciclo $c_t$ sin usar lo del ML

Una vez obtenido el factor $f_t$ (riesgo común), lo descomponemos:
$$
f_t = \tau_t + c_t + \nu_t
$$
donde:
- $\tau_t$: componente persistente (tendencia / régimen de largo plazo),
- $c_t$: componente transitorio (ciclo / mean-reversion),
- $\nu_t$: ruido.

Usamos `UnobservedComponents` con **tendencia local lineal** (level + slope estocásticos) y un ciclo AR.


In [62]:
import pandas as pd
from statsmodels.tsa.statespace.structural import UnobservedComponents

y = f_smooth.copy().sort_index()

# ensure DatetimeIndex
y.index = pd.to_datetime(y.index)

# force a supported frequency (business day grid)
y = y.asfreq("B")     # creates freq="B"; may insert NaNs on missing business days

# optional: handle NaNs (choose ONE)
y = y.ffill()
# y = y.interpolate("time")
# or leave NaNs: state-space can handle missing endog

ucm = UnobservedComponents(
    endog=y,
    level="local linear trend",
    cycle=True,
    stochastic_cycle=True,
)
ucm_res = ucm.fit(disp=False)

print(ucm_res.summary())


c:\Users\herie\miniconda3\envs\a\Lib\site-packages\statsmodels\tsa\filters\hp_filter.py:100: SparseEfficiencyWarning:

spsolve requires A be CSC or CSR matrix format



                        Unobserved Components Results                         
Dep. Variable:      factor_1_smoothed   No. Observations:                 6074
Model:             local linear trend   Log Likelihood               -6737.395
                   + stochastic cycle   AIC                          13484.791
Date:                Sat, 21 Feb 2026   BIC                          13518.346
Time:                        22:48:09   HQIC                         13496.437
Sample:                    11-12-2002                                         
                         - 02-20-2026                                         
Covariance Type:                  opg                                         
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
sigma2.irregular     0.0727      0.004     16.192      0.000       0.064       0.081
sigma2.level         0.4008      0

In [63]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

def arr_a_serie(arr, index, name: str) -> pd.Series:
    """
    Convierte un componente (ndarray o Series/DataFrame) a pd.Series con el índice dado.
    Soporta (T,), (T,1), (1,T).
    """
    if arr is None:
        return None

    # Si ya es Series
    if isinstance(arr, pd.Series):
        s = arr.copy()
        s.index = index
        s.name = name
        return s

    # Si es DataFrame
    if isinstance(arr, pd.DataFrame):
        s = arr.iloc[:, 0].copy()
        s.index = index
        s.name = name
        return s

    # ndarray
    a = np.asarray(arr)

    # Aplana si viene (T,1) o (1,T)
    if a.ndim == 2:
        if a.shape[0] == len(index) and a.shape[1] == 1:
            a = a[:, 0]
        elif a.shape[1] == len(index) and a.shape[0] == 1:
            a = a[0, :]
        else:
            raise ValueError(f"Forma inesperada {a.shape}; no cuadra con T={len(index)}.")
    elif a.ndim != 1:
        raise ValueError(f"Dimensión inesperada {a.ndim}; esperado 1 o 2.")

    if a.shape[0] != len(index):
        raise ValueError(f"Largo {a.shape[0]} != T={len(index)}. Revisa NA/fechas.")

    return pd.Series(a, index=index, name=name)

# --- Extraer componentes ---
idx = f_smooth.index

trend = arr_a_serie(ucm_res.level.smoothed, idx, "tendencia_level")

# slope: a veces existe pero puede ser ndarray o no existir
slope_arr = None
if hasattr(ucm_res, "slope") and ucm_res.slope is not None:
    slope_arr = ucm_res.slope.smoothed if hasattr(ucm_res.slope, "smoothed") else ucm_res.slope
slope = arr_a_serie(slope_arr, idx, "pendiente_slope") if slope_arr is not None else None

# cycle: similar
cycle_arr = None
if hasattr(ucm_res, "cycle") and ucm_res.cycle is not None:
    cycle_arr = ucm_res.cycle.smoothed if hasattr(ucm_res.cycle, "smoothed") else ucm_res.cycle
cycle = arr_a_serie(cycle_arr, idx, "ciclo") if cycle_arr is not None else None



In [64]:
# -------------------------------
# 1) Factor vs tendencia
# -------------------------------
fig1 = go.Figure()
fig1.add_trace(go.Scatter(x=f_smooth.index, y=f_smooth.values, mode="lines", name="factor (suavizado)"))
fig1.add_trace(go.Scatter(x=trend.index,   y=trend.values,   mode="lines", name="tendencia (level)"))

fig1.update_layout(
    title="Factor vs tendencia estimada",
    xaxis_title="Fecha",
    yaxis_title="",
    template="plotly_white",
    legend_title_text="Serie",
)
fig1.show()

# -------------------------------
# 2) Pendiente (si existe)
# -------------------------------
if slope is not None:
    fig2 = go.Figure()
    fig2.add_trace(go.Scatter(x=slope.index, y=slope.values, mode="lines", name="slope"))

    fig2.update_layout(
        title="Pendiente de la tendencia (slope): drift del riesgo",
        xaxis_title="Fecha",
        yaxis_title="",
        template="plotly_white",
    )
    fig2.show()

# -------------------------------
# 3) Ciclo (si existe)
# -------------------------------
if cycle is not None:
    fig3 = go.Figure()
    fig3.add_trace(go.Scatter(x=cycle.index, y=cycle.values, mode="lines", name="cycle"))

    fig3.update_layout(
        title="Ciclo estimado",
        xaxis_title="Fecha",
        yaxis_title="",
        template="plotly_white",
    )
    fig3.show()


## 6) Pronóstico de la tendencia $\widehat{\tau}_{t+h|t}$

El pronóstico del UCM entrega predicción del estado. Como ejemplo, calculamos un forecast del factor y lo comparamos con el histórico.

Para backtesting serio: usar ventana móvil/expandida y comparar RMSE/MAE en horizontes fijos.


In [65]:
# # Forecast objects from statsmodels
h = 52
fc = ucm_res.get_forecast(steps=h)
fc_mean = fc.predicted_mean
fc_ci = fc.conf_int()  # DataFrame with lower/upper columns


In [66]:


# Historical window (last 400 points)
x_hist = f_smooth.index[-400:]
y_hist = f_smooth.values[-400:]

# Forecast + CI
x_fc = fc_mean.index
y_fc = fc_mean.values
y_lo = fc_ci.iloc[:, 0].values
y_hi = fc_ci.iloc[:, 1].values

fig = go.Figure()

# Hist
fig.add_trace(go.Scatter(
    x=x_hist, y=y_hist,
    mode="lines",
    name="factor (hist)"
))

# Forecast line
fig.add_trace(go.Scatter(
    x=x_fc, y=y_fc,
    mode="lines",
    name="forecast (factor)"
))

# Confidence band (two traces: upper then lower with fill='tonexty')
fig.add_trace(go.Scatter(
    x=x_fc, y=y_hi,
    mode="lines",
    line=dict(width=0),
    name="IC (upper)",
    showlegend=False
))
fig.add_trace(go.Scatter(
    x=x_fc, y=y_lo,
    mode="lines",
    line=dict(width=0),
    fill="tonexty",
    name="IC",
    opacity=0.2
))

fig.update_layout(
    title="Pronóstico del factor (con intervalos)",
    xaxis_title="Fecha",
    yaxis_title="",
    template="plotly_white",
    legend_title_text="Serie",
)

fig.show()


## 7) Para el backtest

1. Decide frecuencia: semanal (recomendado) o diaria.
2. Usa $\log$ y estandarización.
3. Ajuste DFM(1 factor). Luego prueba DFM(2 factores) si VIX y MOVE se desacoplan.
4. Extraccion de tendencia $\tau_t$, pendiente $\beta_t$ y marca posibles quiebres (cambios persistentes en $\beta_t$ o saltos en $\tau_t$).
5. Validacion con episodios conocidos de tensión financiera (comparación de fechas) y con backtesting fuera de muestra.


In [ ]:
import numpy as np
import pandas as pd
from statsmodels.tsa.statespace.structural import UnobservedComponents

def _forecast_level_from_states(res, steps: int) -> np.ndarray:
    """
    Pronóstico del estado 'level' usando la recursión del sistema de estados:
      alpha_{t+1} = c + T alpha_t
    Devuelve vector (steps,) con E[level_{T+h} | datos hasta T].
    """
    ssm = res.model.ssm
    Tmat = np.asarray(ssm["transition"])
    c = np.asarray(ssm["state_intercept"]).squeeze()
    alpha = np.asarray(res.filtered_state)[:, -1]  # E[alpha_T | x_1..x_T]

    level_idx = res.model.state_names.index("level")

    out = np.zeros(steps)
    for h in range(steps):
        alpha = c + Tmat @ alpha
        out[h] = alpha[level_idx]
    return out

def rolling_trend_backtest(
    y: pd.Series,
    window: int,
    horizon: int,
    use_cycle: bool = True,
    maxiter: int = 200,
) -> tuple[pd.DataFrame, float]:
    """
    Rolling window backtest:
      - Ajusta UCM en cada ventana
      - Pronostica tendencia (state 'level') a horizon
      - Evalúa vs y real en t+h

    Retorna:
      - df con forecasts/errores
      - RMSE
    """
    y = y.dropna().astype(float)
    y = y.sort_index()

    rows = []
    n = len(y)

    for end in range(window, n - horizon):
        y_train = y.iloc[end - window:end]
        t0 = y_train.index[-1]
        y_true = y.iloc[end + horizon]   # realiza en t0 + horizon (por índice posicional)

        # Ajuste del modelo estructural
        try:
            mod = UnobservedComponents(
                endog=y_train,
                level="local linear trend",
                cycle=use_cycle,
                stochastic_cycle=use_cycle
            )
            res = mod.fit(disp=False, maxiter=maxiter)
        except Exception:
            # Fallback: sin ciclo si hay problemas numéricos
            mod = UnobservedComponents(endog=y_train, level="local linear trend", cycle=False)
            res = mod.fit(disp=False, maxiter=maxiter)

        # Pronóstico de la tendencia (level) a horizon
        level_path = _forecast_level_from_states(res, steps=horizon)
        tau_hat = float(level_path[-1])

        rows.append({
            "origin": t0,
            "target": y.index[end + horizon],
            "tau_hat": tau_hat,
            "y_true": float(y_true),
            "error": float(y_true - tau_hat)
        })

    df_bt = pd.DataFrame(rows).set_index("target")
    rmse = float(np.sqrt(np.mean(df_bt["error"].values**2)))
    return df_bt, rmse

# -----------------------------
# EJEMPLO DE USO
# -----------------------------
# Para backtest realista usa el factor FILTRADO:
y_index = f_filt  # o f_smooth si solo quieres demo (no recomendado para backtest)

# Elige window/horizon según frecuencia:
# - Si semanal: window≈156 (3 años), horizon≈4 (1 mes), o horizon≈12 (3 meses)
# - Si diaria:  window≈500 (2 años hábiles), horizon≈20 (1 mes), o horizon≈60 (3 meses)
df_bt, rmse = rolling_trend_backtest(y_index, window=156, horizon=4, use_cycle=True)

print("RMSE (tendencia vs índice real) =", rmse)
df_bt.head()


In [ ]:
horizons = [1, 4, 12]
rmse_by_h = {}
bt_by_h = {}

for h in horizons:
    df_bt_h, rmse_h = rolling_trend_backtest(y_index, window=156, horizon=h, use_cycle=True)
    bt_by_h[h] = df_bt_h
    rmse_by_h[h] = rmse_h

rmse_table = pd.Series(rmse_by_h, name="RMSE").to_frame()
rmse_table
